# Make AAdict 

In [5]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io

Just run the following cell to open AAdict

In [2]:
# To Reload AAdict without rerunning entire notebook, run this cell
AAdict = pickle.load(open('AAdict','rb'))

In [2]:
# Deborah's function "find_movie"
# Will be used to make aadict

def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            pass
    ## ADDITION TO DEBORAH'S FUNCTION
    # if only one movie left, return
    if len(year_list) == 1:
        return year_list[0]
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                return None
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

In [6]:
# Read in Academy Awards df (AAdf)
AAdf = pd.read_excel("Academy_Awards_2006.xls")

# Update df
# Concat Sort Title with first part of year; store in "title"
# This will be helpful when using ia.search_movie function
AAdf['Year'] = AAdf['Year'].values.astype(str)
AAdf['yr'] = AAdf.apply( lambda row: row['Year'][:4],axis=1 )
AAdf['titleyr'] = AAdf.apply( lambda row: '%s (%s)' % (row['Sort Title'],row['Year'][:4]),axis=1)
# Convert 'Winner' "X" to 1
AAdf['Winner?'] = 1*(AAdf['Winner?'] == 'X')
## TO DO
### Convert 'Country' "0" to USA
# Convert 'Sort Title' to be strings (easier for comparison later)
#AAdf['Sort Title'] = str(AAdf['Sort Title'])
# If movie title is "[no specific film title]", delete
AAdf = AAdf[AAdf['Sort Title'] != "[no specific film title]"].copy()

# Store all possible awards in "awards" list
# This will be used when making aadict to indicate which awards the movie was nominated for/won
awards = list(set(list(AAdf['Category'])))

# Subset the data for easier use when making code
AAsubdf = AAdf[:12]
# View df
AAsubdf.head(5)

,Original Title,English Title,Sort Title,Year,Country,Award,Category,Winner?,Nominee(s),__ Academy Awards,Item,yr,titleyr
0,7th Heaven,7th Heaven,7th Heaven,1927-1928,0,Academy Award,Best Picture,0,Fox,1st,1,1927,7th Heaven (1927)
1,The Racket,The Racket,Racket,1927-1928,0,Academy Award,Best Picture,0,The Caddo Company,1st,2,1927,Racket (1927)
2,Wings,Wings,Wings,1927-1928,0,Academy Award,Best Picture,1,Paramount Famous Lasky,1st,3,1927,Wings (1927)
3,Chang: A Drama of the Wilderness,Chang,Chang: A Drama of the Wilderness,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Paramount Famous Lasky,1st,4,1927,Chang: A Drama of the Wilderness (1927)
4,The Crowd,The Crowd,Crowd,1927-1928,0,Academy Award,Best Unique and Artistic Picture,0,Metro-Goldwyn-Mayer,1st,5,1927,Crowd (1927)


Note:  You can skip the next several cells and just run the next cell (pickle.load) to get the complete moviedict
I ended up doing this in 2 pieces b/c it took long to run on the full file and would sometimes timeout.

In [ ]:
%%time
## ROUND 1: 0-2001

# Get unique list of movie titles from AAdf
AAuniquemovies = list(set(zip(AAdf['English Title'], AAdf['yr'])))
# Keep track of failed movies (i.e. movieobj = None)
AAmissingmovies = list()

AAdict = {}
#for i in range(0,len(AAuniquemovies)):
for i in range(0,2001):
    movieobj = None 
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        int(AAuniquemovies[i][0])
        arg1 = AAuniquemovies[i][0]
        arg11 = str(AAuniquemovies[i][0])
        arg2 = AAuniquemovies[i][1]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i][0])
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
        except ValueError:
            arg1 = AAuniquemovies[i][0]
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    movieobj = ia.search_movie(titleyr)
    if movieobj is None or len(movieobj)>1:
        movieobj = find_movie(arg11, arg2, arg3) # find the movie
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    if movieobj is None:
        arg2 = int(arg2) - 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    
    print (i, AAuniquemovies[i], movieobj)
    
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['English Title']==arg1]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==arg1]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!
    else:
        AAmissingmovies.append((i, AAuniquemovies[i][0], AAuniquemovies[i][1]))

(0, (u'Snow Capers', '1948'), <Movie id:0040807[http] title:_Snow Capers (1948)_>)
(1, (u'Sunbonnet Sue', '1945'), <Movie id:0038135[http] title:_Sunbonnet Sue (1945)_>)
(2, (u'Days of Glory', '1944'), <Movie id:0036748[http] title:_Days of Glory (1944)_>)
(3, (u'Sand', '1949'), <Movie id:0041840[http] title:_Sand (1949)_>)
(4, (u'Red', '1994'), <Movie id:1245526[http] title:_RED (2010)_>)
(5, (u'Road to Morocco', '1942'), <Movie id:0035262[http] title:_Road to Morocco (1942)_>)
(6, (u'The Battle of Gettysburg', '1955'), <Movie id:0047862[http] title:_The Battle of Gettysburg (1955)_>)
(7, (u'Corvette K-225', '1943'), <Movie id:0035757[http] title:_Corvette K-225 (1943)_>)
(8, (u'Witness for the Prosecution', '1957'), <Movie id:0051201[http] title:_Witness for the Prosecution (1957)_>)
(9, (u'The Dead', '1987'), <Movie id:0092843[http] title:_The Dead (1987)_>)
(10, (u"Job's Revolt", '1983'), <Movie id:0085767[http] title:_Jób lázadása (1983)_>)
(11, (u'The Numbers Start With the River

In [9]:
%%time
## ROUND 1: 2001-3500

for i in range(2001,3500):
    movieobj = None 
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        int(AAuniquemovies[i][0])
        arg1 = AAuniquemovies[i][0]
        arg11 = str(AAuniquemovies[i][0])
        arg2 = AAuniquemovies[i][1]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i][0])
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
        except ValueError:
            arg1 = AAuniquemovies[i][0]
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    movieobj = ia.search_movie(titleyr)
    if movieobj is None or len(movieobj)>1:
        movieobj = find_movie(arg11, arg2, arg3) # find the movie
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    if movieobj is None:
        arg2 = int(arg2) - 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    
    print (i, AAuniquemovies[i], movieobj)
    
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['English Title']==arg1]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==arg1]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!
    else:
        AAmissingmovies.append((i, AAuniquemovies[i][0], AAuniquemovies[i][1]))

(2001, (u'Vacation From Marriage', '1946'), <Movie id:0037980[http] title:_Vacation from Marriage (1945)_>)
(2002, (u'The Ninth Circle', '1960'), <Movie id:0053764[http] title:_The Ninth Circle (1960)_>)
(2003, (u'The Wrong Trousers', '1993'), <Movie id:0108598[http] title:_The Wrong Trousers (1993)_>)
(2004, (u'Boom Town', '1940'), <Movie id:0032273[http] title:_Boom Town (1940)_>)
(2005, (u'Enemies, a Love Story', '1989'), <Movie id:0097276[http] title:_Enemies: A Love Story (1989)_>)
(2006, (u'Equus', '1977'), <Movie id:0075995[http] title:_Equus (1977)_>)
(2007, (u'Master and Commander: The Far Side of the World', '2003'), <Movie id:0959939[http] title:_"Scream Play" Master and Commander: The Far Side of the World/Karate Kid/Hulk (2004)_>)
(2008, (u'Splendor in the Grass', '1961'), <Movie id:0055471[http] title:_Splendor in the Grass (1961)_>)
(2009, (u'Ice Station Zebra', '1968'), <Movie id:0063121[http] title:_Ice Station Zebra (1968)_>)
(2010, (u'Mickey and the Seal', '1948'), <

In [10]:
%%time
## ROUND 1: 3500-end

for i in range(3500,len(AAuniquemovies)):
    movieobj = None 
    ## Get arguments needed to search the movie title ##
    # movie title
    try:
        int(AAuniquemovies[i][0])
        arg1 = AAuniquemovies[i][0]
        arg11 = str(AAuniquemovies[i][0])
        arg2 = AAuniquemovies[i][1]  # movie year
        arg3 = ia.search_movie(str(arg1))  # list of possible movies
        titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    except ValueError:
        try:
            arg1 = str(AAuniquemovies[i][0])
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
        except ValueError:
            arg1 = AAuniquemovies[i][0]
            arg11 = arg1
            arg2 = AAuniquemovies[i][1]  # movie year
            arg3 = ia.search_movie(arg1)  # list of possible movies
            titleyr = list(AAdf[AAdf['English Title'] == AAuniquemovies[i][0]]['titleyr'])[0]
    movieobj = ia.search_movie(titleyr)
    if movieobj is None or len(movieobj)>1:
        movieobj = find_movie(arg11, arg2, arg3) # find the movie
    # Could be that movie year is 1 year off (due to the Oscars being biennial in the beginning)
    if movieobj is None:
        arg2 = int(arg2) + 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    if movieobj is None:
        arg2 = int(arg2) - 1
        movieobj = find_movie(arg1, arg2, arg3)
        if (movieobj is None) & (arg3 != []):
            movieobj = arg3[0]
    
    print (i, AAuniquemovies[i], movieobj)
    
    # Had to add this because some movies weren't getting any movies back ("Mighty Mouse in Gypsy Life" #247)
    if movieobj is not None:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['English Title']==arg1]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==arg1]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==arg1) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==arg1]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==arg1]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        ## TO DO, list person/company associated with the award!!
    else:
        AAmissingmovies.append((i, AAuniquemovies[i][0], AAuniquemovies[i][1]))

(3500, (u'Wolfgang', '1997'), <Movie id:0141995[http] title:_Wolfgang (1997)_>)
(3501, (u'The Cliff Dwellers', '1962'), <Movie id:0055853[http] title:_The Cliff Dwellers (1962)_>)
(3502, (u'The Story of Time', '1951'), <Movie id:0044077[http] title:_The Story of Time (1951)_>)
(3503, (u'The Egg and I', '1947'), <Movie id:0039349[http] title:_The Egg and I (1947)_>)
(3504, (u'Love Me or Leave Me', '1955'), <Movie id:0048317[http] title:_Love Me or Leave Me (1955)_>)
(3505, (u"B.F.'s Daughter", '1948'), <Movie id:0040141[http] title:_B.F.'s Daughter (1948)_>)
(3506, (u'Shanks', '1974'), <Movie id:0072149[http] title:_Shanks (1974)_>)
(3507, (u'Three Smart Girls', '1936'), <Movie id:0028373[http] title:_Three Smart Girls (1936)_>)
(3508, (u'Zelig', '1983'), <Movie id:0086637[http] title:_Zelig (1983)_>)
(3509, (u'To Forget Venice', '1979'), <Movie id:0079054[http] title:_To Forget Venice (1979)_>)
(3510, (u'Two Women', '1961'), <Movie id:0054749[http] title:_Two Women (1960)_>)
(3511, (u"

### Check
Check to see what movies are missing from AAdict compared to the unique movies from AAdf

In [14]:
# CHECK
print "number of movies in AAdict:", len(AAdict.keys())
print "number of movies in AAdf:", len(AAuniquemovies)
    # inconsistent numbers...
    
## DEBUG ##
# Get list of what is in the dictionary & what movies returned "None"
debuglist = list()
for key in AAdict.keys():
    debuglist.append((AAdict[key]['title'], AAdict[key]['year']))

# Compare to list of movies from AAdf
print "list of movies missing from AAdict:"
missing = list(set(AAuniquemovies) -set(debuglist))
print len(missing)
missing

number of movies in AAdict: 4158
number of movies in AAdf: 4209
list of movies missing from AAdict:
119


[(u'Song without End (The Story of Franz Liszt)', '1960'),
 (u'Amarcord', '1975'),
 (u'Van Gogh', '1949'),
 (u'First Love', '1970'),
 (u'The Old Man and the Sea', '1999'),
 (u"My Night at Maud's", '1970'),
 (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)',
  '1946'),
 (u'Sweet and Lowdown', '1999'),
 (u'Men in Black', '1997'),
 (u'A Voice Is Born: The Story of Niklos Gafni', '1947'),
 (u"Jacques-Yves Cousteau's World Without Sun", '1964'),
 (u'The Time Machine', '2002'),
 (u'The Forbidden Volcano', '1966'),
 (u'Good Fellas', '1990'),
 (u'Skyscraper', '1959'),
 (u'A Farewell to Arms', '1957'),
 (u'War and Peace', '1968'),
 (u'In Which We Serve', '1943'),
 (u'Hamlet', '1990'),
 (u'The War of the Worlds', '1953'),
 (u'Heaven Can Wait', '1978'),
 (u'The Champ', '1979'),
 (u'Investigation of a Citizen Above Suspicion', '1971'),
 (u'Day for Night', '1974'),
 (u'Contact', '1997'),
 (u'Richard III', '1995'),
 (u'Marriage Italian Style', '1965'),
 (u'Goodbye, Mr. Chips'

###Debug

####DEBUG, STEP 1
Adjust 1 year up/down and research; Save the remaining missing movies in AAmissingmovies1

In [29]:
%%time

AAmissingmovies1 = list()
for missingmovie in missing:
    movie = find_movie( missingmovie[0], missingmovie[1], ia.search_movie(missingmovie[0]))
    if movie is None:
        movie = find_movie( missingmovie[0], int(missingmovie[1]) - 1, ia.search_movie(missingmovie[0]))
    if movie is None:
        movie = find_movie( missingmovie[0], int(missingmovie[1]) + 1, ia.search_movie(missingmovie[0]))
    if movie is not None:
        movid = movie.movieID
        title = missingmovie[0]
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['English Title']==title]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==title]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==title) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==title]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==title]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        print ("True", missingmovie, movie)
    else:
        AAmissingmovies1.append(missingmovie)
        print ("False", missingmovie, movie)

('False', (u'Song without End (The Story of Franz Liszt)', '1960'), None)
('True', (u'Amarcord', '1975'), <Movie id:2038256[http] title:_Diario segreto di Amarcord (1974)_>)
('True', (u'Van Gogh', '1949'), <Movie id:0040930[http] title:_Van Gogh (1948)_>)
('True', (u'First Love', '1970'), <Movie id:0065703[http] title:_First Love (1970)_>)
('True', (u'The Old Man and the Sea', '1999'), <Movie id:0207639[http] title:_The Old Man and the Sea (1999)_>)
('True', (u"My Night at Maud's", '1970'), <Movie id:0064612[http] title:_My Night at Maud's (1969)_>)
('False', (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)', '1946'), None)
('True', (u'Sweet and Lowdown', '1999'), <Movie id:0158371[http] title:_Sweet and Lowdown (1999)_>)
('False', (u'Men in Black', '1997'), None)
('False', (u'A Voice Is Born: The Story of Niklos Gafni', '1947'), None)
('False', (u"Jacques-Yves Cousteau's World Without Sun", '1964'), None)
('True', (u'The Time Machine', '2002'), <Movie id:026869

In [33]:
print "Movies still missing:", len(AAmissingmovies1)

Movies still missing: 26


#### DEBUG, STEP 2
Search using non-English title; Save the remaining missing movies in AAmissingmovies2

In [36]:
%%time

# See if non English Title is searchable
AAmissingmovies2 = list()
for movie in AAmissingmovies1:
    movieobj = None
    movietitle = list(AAdf[AAdf['English Title']==movie[0]]['Sort Title'])[0]
    movieyr = list(AAdf[AAdf['English Title']==movie[0]]['yr'])[0]
    movieobj = ia.search_movie((movietitle + " (%s)" %movieyr))
    if movieobj is None:
        movieyr = int(movieyr)
        movieobj = find_movie(movietitle, movieyr - 1, ia.search_movie(movietitle))
    if movieobj is None:
        movieobj = find_movie(movietitle, movieyr + 1, ia.search_movie(movietitle))
    if len(movieobj) >= 1:
        movieobj = movieobj[0]
        movid = movieobj.movieID
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['Sort Title']==movietitle]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['Sort Title']==movietitle]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['Sort Title']==movietitle) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['Sort Title']==movietitle]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['Sort Title']==movietitle]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])   
        print ("True", movie, movieobj)
    else:
        AAmissingmovies2.append(movie)
        print ("False", movie, movieobj)

('True', (u'Song without End (The Story of Franz Liszt)', '1960'), <Movie id:0054324[http] title:_Song Without End (1960)_>)
('True', (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)', '1946'), <Movie id:0038825[http] title:_Paramount News Issue #37 (1946)_>)
('True', (u'Men in Black', '1997'), <Movie id:0025488[http] title:_Men in Black (1934)_>)
('False', (u'A Voice Is Born: The Story of Niklos Gafni', '1947'), [])
('True', (u"Jacques-Yves Cousteau's World Without Sun", '1964'), <Movie id:0058364[http] title:_World Without Sun (1964)_>)
('True', (u'The Forbidden Volcano', '1966'), <Movie id:0061163[http] title:_Le volcan interdit (1966)_>)
('True', (u'War and Peace', '1968'), <Movie id:0049934[http] title:_War and Peace (1956)_>)
('True', (u'Anastasia', '1997'), <Movie id:0048947[http] title:_Anastasia (1956)_>)
('True', (u'The Brothers Karamazov', '1969'), <Movie id:0051435[http] title:_The Brothers Karamazov (1958)_>)
('True', (u'Forbidden Games', '1954'), <

In [42]:
print "Movies still missing:", len(AAmissingmovies2)

Movies still missing: 9


#### DEBUG, STEP 3
Mannually search remaining movies

In [41]:
missingids = {}
missingids['A Voice Is Born: The Story of Niklos Gafni'] = '0039968'
missingids['Mighty Mouse in Gypsy Life'] = '0037912'
missingids['Cavalcade of Dance with Veloz and Yolanda'] = '0035724'
missingids['The Usual Unidentified Thieves'] = '0052216'
missingids["Eyes on the Prize: America's Civil Rights Years/Bridge to Freedom 1965"] = '0092999'
missingids['Sandokan No. 8'] = '0073654'
missingids["Screen Snapshots' 50th Anniversary of Motion Pictures"] = '0036576'
missingids['Mama Turns a Hundred'] = '0079517'
missingids['Rashomon'] = '0042876'

In [48]:
for movie in missingids:
    movid = missingids[movie]
    AAdict[movid] = {}
    # "title": title of movie
    AAdict[movid]['title'] = movie
    # "nominations": list of Oscar nominations
    AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==movie]['Category'])
    # "won": list of Oscars won
    AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==movie) & (AAdf['Winner?']==1)]['Category'])
    # "year": year Oscar won
    AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==movie]['yr'])[0]
    # "country": country of movie
    AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==movie]['Country'])[0]
    # make each award individual key and the value to indicate whether movie won/nominated or not
    # Loop through awards list and indicate if movie was nominated or won
    for award in awards:
        # "Nominated award_name": True or False
        AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
        # "Nominated award_name": True or False
        AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])        

In [ ]:
# To Save new AAdict, run this cell
#AAdict = pickle.load(open('AAdict','rb'))

In [7]:
AAdict['0120338']['won']

[u'Best Writing, Original Screenplay',
 u'Best Picture',
 u'Best Director',
 u'Best Music, Dramatic Score',
 u'Best Music, Song',
 u'Best Film Editing',
 u'Best Cinematography',
 u'Best Art Direction',
 u'Best Costume Design',
 u'Best Visual Effects',
 u'Best Sound',
 u'Best Sound Effects Editing']

In [452]:
#missingids = {}
#missingids['The Forbidden Volcano'] = '0061163'
#missingids['The Usual Unidentified Thieves'] = '0052216'
#missingids['Kate & Leopold'] = '0035423'
#missingids['Malcolm X'] = '0104797'
#missingids['Cavalcade of Dance with Veloz and Yolanda'] = '0035724'
#missingids['A Voice Is Born: The Story of Niklos Gafni'] = '0039968'
#missingids["Eyes on the Prize: America's Civil Rights Years/Bridge to Freedom 1965"] = '0092999'
#missingids['Sandokan No. 8'] = '0073654'
#missingids["Screen Snapshots' 50th Anniversary of Motion Pictures"] = '0036576'
#missingids['Song without End (The Story of Franz Liszt)'] = '0054324'
#missingids['Mighty Mouse in Gypsy Life'] = '0037912'
#missingids['The Sorrow and the Pity'] = '0066904'
#missingids['Mama Turns a Hundred'] = '0079517'
#missingids['Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)'] = '0038825'
#missingids['Men in Black'] = '0119654'
#missingids["Jacques-Yves Cousteau's World Without Sun"] = '0058364'
#missingids['War and Peace'] = '0063794'
#missingids['Yes, Giorgio'] = '0084931'
#missingids['Rashomon'] = '0042876 '
#missingids['Henry V'] = '0036910'